## 7.1 데커레이터 기본 지식

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner # deco()가 inner() 함수 객체를 반환한다. 

In [3]:
@deco
def target(): # target()을 deco로 데커레이트했다. 
    print('running target()')

In [4]:
target() # 데커레이트된 target()을 호출하면 실제로는 inner()를 실행한다. 

running inner()


In [5]:
target # target이 inner()를 가리키고 있다. 

<function __main__.deco.<locals>.inner()>

## 7.2. 파이썬이 데커레이터를 실행하는 시점

In [6]:
registry = []  # @register로 데커레이트된 함수들에 대한 참조를 담는다. 

# 함수를 인수로 받는다. 
def register(func): 
    print(f'running register({func})')
    registry.append(func)

    # 반드시 함수를 반환해야 한다. (인수 그대로 반환)
    return func 


@register  # @register로 장식 되었다. 
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # 데커레이트 x 
    print('running f3()')

def main():  
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main() 

running register(<function f1 at 0x104868790>)
running register(<function f2 at 0x1046eaf70>)
running main()
registry -> [<function f1 at 0x104868790>, <function f2 at 0x1046eaf70>]
running f1()
running f2()
running f3()


## 7.3 데커레이터로 개선한 전략 패턴

In [8]:
promos = []

# promo_func를 promos 리스트에 추가한 후 그대로 반환한다. 
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

# @promotion으로 데커레이트한 함수는 모두 promos 리스트에 추가된다. 
@promotion  
def fidelity(order):
    """충성도 포인트가 1,000점 이상인 고객은 전체 주문에 대해 5% 할인을 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """하나의 주문에서 20개 이상의 동일 상품을 구입하면 해당 상품에 대해 10% 할인을 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """서로 다른 상품을 10종류 이상 주문하면 전체 주문에 대해 7% 할인을 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

# best_promo()는 promos 리스트에 의존하므로 변경할 필요 없다. 
def best_promo(order):  
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)


## 7.4. 변수 범위 규칙

In [9]:
def f1(a):
    print(a)
    print(b)

In [10]:
f1(3)

3


NameError: name 'b' is not defined

In [11]:
b = 6
f1(3)

3
6


In [12]:
b = 6

def f2(a):
    print(a)
    print(b)
    b = 9

In [13]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [15]:
b = 6

def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [16]:
b

9

In [17]:
f3(3)

3
9


In [18]:
b = 30
b

30

In [19]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [20]:
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 7.5 클로저

In [22]:
# 이동 평균을 계산하는 클래스

class Averager:
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [29]:
avg = Averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [30]:
# 이동 평균을 계산하는 고위 함수
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

In [31]:
avg = make_averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [32]:
avg.__code__.co_varnames

('new_value', 'total')

In [33]:
avg.__code__.co_freevars

('series',)

In [34]:
avg.__closure__

(<cell at 0x104b02b20: list object at 0x1047c9980>,)

In [35]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## 7.6 nonlocal 선언

In [36]:
# 전체 이력을 유지하지 않고 이동 평균을 계산하는 잘못된 고위 함수
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [37]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [38]:
# 전체 이력을 유지하지 않고 이동 평균 계산하기(nonlocal로 수정)
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

## 7.7 간단한 데커레이터 구현하기

In [83]:
import time

def clock(func):
    # 임의 개수의 위치 인수를 받을 수 있도록 정의
    def clocked(*args): 
        t0 = time.perf_counter()
    
        # clocked()에 대한 클로저에 자유 변수 func이 들어가야 코드가 작동한다. 
        result = func(*args)  
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        
        print(f'[{elapsed:.8f}s] {name}({arg_str}) -> {result}]')
        return result
     
    # 내부 함수를 반환해서 데커레이트왼 함수를 대체한다. 
    return clocked

In [84]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling snooze(.6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12704829s] snooze(0.123) -> None]
**************************************** Calling snooze(.6)
[0.00000104s] factorial(1) -> 1]
[0.00003071s] factorial(2) -> 2]
[0.00004846s] factorial(3) -> 6]
[0.00006388s] factorial(4) -> 24]
[0.00008087s] factorial(5) -> 120]
[0.00009842s] factorial(6) -> 720]
6! = 720


In [78]:
factorial.__name__

'clocked'

In [85]:
import time
import functools


def clock(func):
    
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = [f'{k}={w}' for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
            
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:.6f}s] {name}({arg_str}) -> {result}]')
        return result
    return clocked

In [86]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling snooze(.6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.128066s] snooze(0.123) -> None]
**************************************** Calling snooze(.6)
[0.000001s] factorial(1) -> 1]
[0.000034s] factorial(2) -> 2]
[0.000053s] factorial(3) -> 6]
[0.000071s] factorial(4) -> 24]
[0.000089s] factorial(5) -> 120]
[0.000109s] factorial(6) -> 720]
6! = 720


## 7.8 표준 라이브러리에서 제공하는 데커레이터

In [93]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(30))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




[0.000001s] fibonacci(1) -> 1]
[0.000000s] fibonacci(0) -> 0]
[0.000001s] fibonacci(1) -> 1]
[0.000005s] fibonacci(2) -> 1]
[0.000016s] fibonacci(3) -> 2]
[0.009504s] fibonacci(4) -> 3]
[0.009515s] fibonacci(5) -> 5]
[0.009535s] fibonacci(6) -> 8]
[0.009567s] fibonacci(7) -> 13]
[0.009616s] fibonacci(8) -> 21]
[0.009697s] fibonacci(9) -> 34]
[0.009826s] fibonacci(10) -> 55]
[0.010035s] fibonacci(11) -> 89]
[0.010374s] fibonacci(12) -> 144]
[0.000001s] fibonacci(1) -> 1]
[0.000000s] fibonacci(0) -> 0]
[0.000000s] fibonacci(1) -> 1]
[0.000005s] fibonacci(2) -> 1]
[0.000009s] fibonacci(3) -> 2]
[0.000000s] fibonacci(0) -> 0]
[0.000000s] fibonacci(1) -> 1]
[0.000004s] fibonacci(2) -> 1]
[0.000000s] fibonacci(1) -> 1]
[0.000000s] fibonacci(0) -> 0]
[0.000000s] fibonacci(1) -> 1]
[0.000004s] fibonacci(2) -> 1]
[0.000008s] fibonacci(3) -> 2]
[0.000015s] fibonacci(4) -> 3]
[0.000028s] fibonacci(5) -> 5]
[0.000000s] fibonacci(0) -> 0]
[0.000000s] fibonacci(1) -> 1]
[0.000004s] fibonacci(2) -> 

In [94]:
import functools

# lru_cache() 데커레이터는 설정 매개변수를 받기 때문에 일반 함수처럼 호출해야 한다. 
@functools.lru_cache() 
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(30))

[0.000002s] fibonacci(0) -> 0]
[0.000002s] fibonacci(1) -> 1]
[0.000550s] fibonacci(2) -> 1]
[0.000002s] fibonacci(3) -> 2]
[0.000586s] fibonacci(4) -> 3]
[0.000002s] fibonacci(5) -> 5]
[0.000618s] fibonacci(6) -> 8]
[0.000002s] fibonacci(7) -> 13]
[0.000648s] fibonacci(8) -> 21]
[0.000001s] fibonacci(9) -> 34]
[0.000680s] fibonacci(10) -> 55]
[0.000002s] fibonacci(11) -> 89]
[0.000712s] fibonacci(12) -> 144]
[0.000002s] fibonacci(13) -> 233]
[0.000743s] fibonacci(14) -> 377]
[0.000001s] fibonacci(15) -> 610]
[0.000777s] fibonacci(16) -> 987]
[0.000002s] fibonacci(17) -> 1597]
[0.000807s] fibonacci(18) -> 2584]
[0.000002s] fibonacci(19) -> 4181]
[0.000846s] fibonacci(20) -> 6765]
[0.000001s] fibonacci(21) -> 10946]
[0.000879s] fibonacci(22) -> 17711]
[0.000001s] fibonacci(23) -> 28657]
[0.000910s] fibonacci(24) -> 46368]
[0.000001s] fibonacci(25) -> 75025]
[0.000940s] fibonacci(26) -> 121393]
[0.000002s] fibonacci(27) -> 196418]
[0.000970s] fibonacci(28) -> 317811]
[0.000001s] fibonacc

In [104]:
import html
import numbers
from collections import abc
from functools import singledispatch

# singledispatch는 객체형을 다룰 기반 함수를 표시한다. 
@singledispatch  
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

# 각각의 특화된 함수는 @<기반_함수>.register(<객체형>)으로 데커레이트된다. 
@htmlize.register(str)
# 특화된 함수의 이름은 필요 없으므로 언더바로 함수명을 지정한다. 
def _(text):  
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

# 특별하게 처리할 자료형을 추가할 때마다 새로운 함수를 등록한다. 
# numbers.Integral은 int의 가상 슈퍼클래스다.
@htmlize.register(numbers.Integral)  
def _(n):
    return f'<pre>{n} (0x{n:x})</pre>'

# 동일한 함수로 여러 자료형을 지원하기 위해 register 데커레이터를 여러 개 쌓아올릴 수 있다.
@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [105]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [106]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [107]:
htmlize("Helmlich & Co. \n- a game")

'<p>Helmlich &amp; Co. <br/>\n- a game</p>'

In [108]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [109]:
print(htmlize(['alpha', 42, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>42 (0x2a)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## 7.10 매개변수화된 데커레이터

In [110]:
# 함수의 추가와 제거를 빠르게 하기 위해 registry를 집합형으로 정의한다. 
registry = set()  

# register()는 선택적 키워드 인수를 받는다. 
def register(active=True):  
    # decorate() 내부 함수가 실제 데커레이터다.
    def decorate(func):  
        print('running register'
              f'(active={active})->decorate({func})')
        if active:   # 클로저에서 읽어온 active 인수가 True일 때만 func를 등록한다. 
            registry.add(func)
        else:
            registry.discard(func)  # active가 True가 아니고 func가 registry에 들어있으면 제거한다. 

        return func  # decorate()는 데커레이터이므로 함수를 반환한다. 
    return decorate  # register()는 데커레이터 팩토리이므로 decorate()를 반환한다. 

@register(active=False)  # @register 팩토리는 원하는 매개변수와 함께 함수로 호출해야 한다. 
def f1():
    print('running f1()')

@register()  # 인수를 전달하지 않더라도 register는 여전히 함수로 호출해야 하므로 @register() 형태로 호출한다. 
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x105d10d30>)
running register(active=True)->decorate(<function f2 at 0x105d10e50>)


In [113]:
import registration_param

running register(active=False)->decorate(<function f1 at 0x105d100d0>)
running register(active=True)->decorate(<function f2 at 0x105d10940>)


In [114]:
registration_param.registry

{<function registration_param.f2()>}

In [115]:
from registration_param import *

In [116]:
registry

{<function registration_param.f2()>}

In [117]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x105d10af0>)


<function registration_param.f3()>

In [118]:
registry

{<function registration_param.f2()>, <function registration_param.f3()>}

In [119]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x105d10940>)


<function registration_param.f2()>

In [120]:
registry

{<function registration_param.f3()>}

In [121]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # 매개변수화된 데커레이터 팩토리
    def decorate(func):      # decorate()가 실제 데커레이터이다. 
        def clocked(*_args): # clocked()는 데커레이트된 함수를 래핑한다. 
            t0 = time.perf_counter()
            _result = func(*_args)  # 데터레이트된 함수의 실제 결과를 _result에 저장한다. 
            elapsed = time.perf_counter() - t0
            name = func.__name__

            # _args가 실제 clocked()의 인수를 담고 있으며, args는 출력하기 위한 문자열이다. 
            args = ', '.join(repr(arg) for arg in _args)  
            
            # result는 출력하기 위해 _result를 문자열로 표현한 것
            result = repr(_result)  
            
            # **locals()를 사용하면 fmt가 clocked()의 지역 변수를 모두 참조할 수 있게 해준다. 
            print(fmt.format(**locals()))  
            
            # clocked()는 데커레이트된 함수를 대체하므로, 원래 함수가 반환하는 값을 반환해야 한다. 
            return _result 
        
        # decorate()는 clocked()를 반환한다.
        return clocked
    
    # clock()은 decorate()를 반환한다. 
    return decorate

if __name__ == '__main__':

    # 적용된 데커레이터는 기본 포맷 문자열을 사용한다. 
    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12482771s] snooze(0.123) -> None
[0.12724296s] snooze(0.123) -> None
[0.12803525s] snooze(0.123) -> None
